# Tests for DataMux

In [1]:
import asyncio
import websockets
import logging

In [4]:
import json
from serializers import Serializer
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)

serializer = Serializer("json")
async def test():
    async with websockets.connect('ws://127.0.0.1:3300/ws') as websocket:
        await websocket.send(b"list_collections|")
        payload = await websocket.recv()
        topic, content = serializer.decode(payload)
        print(topic, content)

In [5]:
loop = asyncio.get_event_loop()
loop.create_task(test())

<Task pending name='Task-9' coro=<test() running at /tmp/ipykernel_67367/121792762.py:6>>

b'list_collections' {'collections': []}
